In [1]:
import os, sys, time
from time import sleep
from pathlib import Path
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [2]:
df_metrics_data = pd.read_csv("C:/data/inbound/2020-03-20/metric_data.csv")

C:\Users\derbates\miniconda3\envs\py3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,9,12,20,25,30,31,59,74,79,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# CHANGE 'PfBatchID' to STRING
# Change data type of column 'Age' from int64 to string i.e. object type
df_metrics_data['PfBatchID'] = df_metrics_data['PfBatchID'].astype('object')
df_metrics_data['ProductID'] = df_metrics_data['ProductID'].astype('object')

In [4]:
df_metrics_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85104 entries, 0 to 85103
Data columns (total 95 columns):
Unnamed: 0          85104 non-null int64
ProductID           85104 non-null object
ProductNo           85104 non-null object
ProductName         85096 non-null object
LineID              85104 non-null float64
Description         21554 non-null object
Specials            5731 non-null object
OriginationDate     84931 non-null object
EntryDate           85104 non-null object
Status              22416 non-null object
MatrixNumber        85104 non-null object
Components          85104 non-null float64
Availability        18098 non-null object
EquivalentSoln      0 non-null float64
EquivalentNeat      0 non-null float64
Packaging           84050 non-null object
Storage             84928 non-null object
ExpirationDate      85104 non-null float64
ShipType            80418 non-null object
UN#                 73535 non-null object
ShipClass           72390 non-null object
PackGrp       

In [5]:
# << set column to datetime >>
df_metrics_data['QCDate'] = pd.to_datetime(df_metrics_data['QCDate'])

In [6]:
# << set column to datetime >>
df_metrics_data['AmpDate'] = pd.to_datetime(df_metrics_data['AmpDate'])
df_metrics_data['OriginationDate'] = pd.to_datetime(df_metrics_data['OriginationDate'])
df_metrics_data['EntryDate'] = pd.to_datetime(df_metrics_data["EntryDate"])

### only look at data for `specific time range`

In [7]:
# Select observations between two datetimes
# 2019-11-01
# to:
# 2020-03-01
df_metrics_obv = df_metrics_data[(df_metrics_data['QCDate'] > '2019-11-01') & (df_metrics_data['QCDate'] <= '2020-03-01')]

### Total PRODUCTS tested PER WEEK PER PRODUCT TYPE

In [8]:
df_cycle_time = pd.DataFrame(data=df_metrics_obv.groupby(['QCDate', 'ProductLevel'])['PfBatchID']) # ProductNo

In [9]:
df_cycle_time.to_csv(os.path.join("c:/data/outbound/2020-03-24/"
                                     + str(pd.Timestamp.now())[:10]
                                     + "-cycle-groupby.csv"), index=True)

In [10]:
df_cycle_groupby = pd.DataFrame(data=df_metrics_obv.groupby(['QCDate', 'ProductLevel'])[['ProductID']])

In [11]:
df_cycle_groupby

,0,1
0,"(2019-11-04 00:00:00, 0.0)",Unnamed: 0 ProductID ProductNo Pr...
1,"(2019-11-04 00:00:00, 1.0)",Unnamed: 0 ProductID ProductNo ...
2,"(2019-11-04 00:00:00, 2.0)",Unnamed: 0 ProductID ProductNo ...
3,"(2019-11-04 00:00:00, 3.0)",Unnamed: 0 ProductID ProductNo ...
4,"(2019-11-05 00:00:00, 1.0)",Unnamed: 0 ProductID ProductNo ...
...,...,...
209,"(2020-02-27 00:00:00, 1.0)",Unnamed: 0 ProductID ProductNo \ ...
210,"(2020-02-27 00:00:00, 2.0)",Unnamed: 0 ProductID ProductNo ...
211,"(2020-02-28 00:00:00, 0.0)",Unnamed: 0 ProductID ProductNo \ 8045...
212,"(2020-02-28 00:00:00, 1.0)",Unnamed: 0 ProductID ProductNo \ ...


### Determine `days in QC`

In [12]:
df_days_in_QC = pd.DataFrame(data=df_metrics_obv['PfBatchID'])

In [13]:
# CREATE new DATAFRAME with just PFBatchID numbers
#df_days_in_QC = pd.DataFrame(data=df_metrics_obv['PfBatchID'])

In [14]:
calculation = df_metrics_obv['QCDate'] - df_metrics_obv['AmpDate']

In [15]:
df_days_in_QC['time-in-QC'] = calculation

In [16]:
df_days_in_QC['ProductLevel'] = df_metrics_obv['ProductLevel']

In [17]:
df_days_in_QC.agg({'time-in-QC':'mean'})

time-in-QC   7 days 21:28:12.715231
dtype: timedelta64[ns]

In [18]:
df_days_in_QC.to_csv("C:/data/Outbound/2020-03-24/" + str(pd.Timestamp.now())[:10] + "-days-in-QC.csv")

In [19]:
# GROUPBY TEST?
df_metrics_obv.groupby(['QCDate', 'ProductLevel'])['PfBatchID'].count()

QCDate      ProductLevel
2019-11-04  0.0              1
            1.0              3
            2.0             13
            3.0              2
2019-11-05  1.0             10
                            ..
2020-02-27  1.0             19
            2.0              6
2020-02-28  0.0              1
            1.0             20
            2.0              7
Name: PfBatchID, Length: 214, dtype: int64

In [20]:
df_metrics_obv.groupby(['QCDate', 'ProductLevel'])[['ProductID']]

### How many `PfBatchID`, per `ProductLevel`, per `QCDate`?

In [21]:
df_metrics_obv.groupby(['QCDate', 'ProductLevel'])[['PfBatchID']].count()

PfBatchID
QCDate     ProductLevel           
2019-11-04 0.0                   1
           1.0                   3
           2.0                  13
           3.0                   2
2019-11-05 1.0                  10
...                            ...
2020-02-27 1.0                  19
           2.0                   6
2020-02-28 0.0                   1
           1.0                  20
           2.0                   7

[214 rows x 1 columns]

In [22]:
df_metrics_obv.groupby(['QCDate', 'ProductLevel'])[['PfBatchID']].count().to_csv("C:/data/outbound/2020-03-24/df-obversation-groupby-" 
                                                                                 + str(pd.Timestamp.now())[:10]
                                                                                 + ".csv")

In [23]:
df_metrics_obv.groupby(['QCDate', 'ProductLevel'])[['PfBatchID']].sum()

PfBatchID
QCDate     ProductLevel                                                   
2019-11-04 0.0                                                    NAA06719
           1.0                                    NAA06711NAA06710NAA06689
           2.0           NAA06560NAA06658NAA06661NAA06635NAA06638NAA066...
           3.0                                            NAA06366NAA06648
2019-11-05 1.0           NAA06673NAA06728NAA06737NAA06686NAA06700NAA067...
...                                                                    ...
2020-02-27 1.0           NAB01057NAB01062NAB01067NAB01068NAB01044NAB010...
           2.0            NAB00967NAB01082NAB01020NAB01023NAB01019NAB01046
2020-02-28 0.0                                                    NAB01115
           1.0           NAB01085NAB01130NAB01132NAB01134NAB01121NAB010...
           2.0           NAB01074NAB01073NAB00975NAB01077NAB01075NAB010...

[214 rows x 1 columns]

In [24]:
# set index
df_metrics_test = df_metrics_data.set_index(['QCDate', 'ProductLevel'])

In [25]:
df_metrics_test

Unnamed: 0 ProductID      ProductNo  \
QCDate     ProductLevel                                        
2012-09-05 2.0                    5      5420    01080-68703   
2013-10-31 2.0                    6      5420    01080-68703   
2012-04-26 2.0                    7      5420    01080-68703   
           2.0                    8      5420    01080-68703   
2009-12-04 2.0                    9      5420    01080-68703   
...                             ...       ...            ...   
2012-01-17 2.0               104249     50108  ZX-TESTMIX-01   
2016-03-11 2.0               104250     50108  ZX-TESTMIX-01   
2018-02-20 2.0               104253     50108  ZX-TESTMIX-01   
           2.0               104254     50108  ZX-TESTMIX-01   
2019-06-10 2.0               104255     50108  ZX-TESTMIX-01   

                                      ProductName  LineID Description  \
QCDate     ProductLevel                                                 
2012-09-05 2.0                    Gradient Sample     7.0         NaN   
2013-10-31 2.0                    Gradient Sample     7.0         NaN   
2012-04-26 2.0                    Gradient Sample     7.0         NaN   
           2.0                    Gradient Sample     7.0         NaN   
2009-12-04 2.0                    Gradient Sample     7.0         NaN   
...                                           ...     ...         ...   
2012-01-17 2.0           GC x GC Testing Solution     7.0         NaN   
2016-03-11 2.0           GC x GC Testing Solution     7.0         NaN   
2018-02-20 2.0           GC x GC Testing Solution     7.0         NaN   
           2.0           GC x GC Testing Solution     7.0         NaN   
2019-06-10 2.0           GC x GC Testing Solution     7.0         NaN   

                            Specials OriginationDate  EntryDate Status  ...  \
QCDate     ProductLevel                                                 ...   
2012-09-05 2.0           01080-68703      1998-02-04 2011-05-09    oem  ...   
2013-10-31 2.0           01080-68703      1998-02-04 2011-05-09    oem  ...   
2012-04-26 2.0           01080-68703      1998-02-04 2011-05-09    oem  ...   
           2.0           01080-68703      1998-02-04 2011-05-09    oem  ...   
2009-12-04 2.0           01080-68703      1998-02-04 2011-05-09    oem  ...   
...                              ...             ...        ...    ...  ...   
2012-01-17 2.0                   NaN      2011-04-19 2011-05-09    NaN  ...   
2016-03-11 2.0                   NaN      2011-04-19 2011-05-09    NaN  ...   
2018-02-20 2.0                   NaN      2011-04-19 2011-05-09    NaN  ...   
           2.0                   NaN      2011-04-19 2011-05-09    NaN  ...   
2019-06-10 2.0                   NaN      2011-04-19 2011-05-09    NaN  ...   

                        QCValidation  FgInvCount  \
QCDate     ProductLevel                            
2012-09-05 2.0                   2.0      2600.0   
2013-10-31 2.0                   2.0      2123.0   
2012-04-26 2.0                   2.0      3400.0   
           2.0                   2.0      1563.0   
2009-12-04 2.0                   2.0      1156.0   
...                              ...         ...   
2012-01-17 2.0                   2.0       129.0   
2016-03-11 2.0                   2.0       602.0   
2018-02-20 2.0                   2.0       366.0   
           2.0                   2.0       320.0   
2019-06-10 2.0                   2.0       320.0   

                                                   FgAccpacNote  CofAHeaderNo  \
QCDate     ProductLevel                                                         
2012-09-05 2.0                                              NaN           2.0   
2013-10-31 2.0                                              NaN           2.0   
2012-04-26 2.0                                              NaN           2.0   
           2.0                                          back up           2.0   
2009-12-04 2.0                zero out ce-3778 (139

In [26]:
# check out multi-index?
df_metrics_test.index

MultiIndex([('2012-09-05', 2.0),
            ('2013-10-31', 2.0),
            ('2012-04-26', 2.0),
            ('2012-04-26', 2.0),
            ('2009-12-04', 2.0),
            ('2008-07-02', 2.0),
            ('2005-03-02', 2.0),
            ('2003-08-21', 2.0),
            ('2003-08-05', 2.0),
            ('2002-08-12', 2.0),
            ...
            ('2018-04-06', 2.0),
            ('2017-06-14', 1.0),
            ('2017-07-03', 1.0),
            ('2012-01-19', 2.0),
            ('2013-10-17', 2.0),
            ('2012-01-17', 2.0),
            ('2016-03-11', 2.0),
            ('2018-02-20', 2.0),
            ('2018-02-20', 2.0),
            ('2019-06-10', 2.0)],
           names=['QCDate', 'ProductLevel'], length=85104)

In [27]:
# sort index
df_metrics_test.sort_index(inplace=True)

In [28]:
# check out columns son
for column in df_metrics_test.columns: # {
    print(column)
# }

Unnamed: 0
ProductID
ProductNo
ProductName
LineID
Description
Specials
OriginationDate
EntryDate
Status
MatrixNumber
Components
Availability
EquivalentSoln
EquivalentNeat
Packaging
Storage
ExpirationDate
ShipType
UN#
ShipClass
PackGrp
ShipSpecial
Hazard1
Hazard2
Hazard3
Prop65
MSDS
Analysis
CofAHeader
Agilent#
Tedia#
ProdNotes
MatrixNotes
RecipeId
RushPriority
IsVoided
SagePartNumber
DateInserted
CofATemplate
UniqueRequestID
Test_Timestamp
PfIDNo
PfBatchID
OrdID
QuoteNo
PfSentTo
PrepDate
PrepVolume
PrepUnit
PrepVessel
PrepVBarcode
PrdSaleUnit
PrepMatrixNo
PrepMatrixLot
PrepInits
PrepMemo
PrepNotebookRef
BulkQCStatus
BulkQCDate
BulkPassFail
BulkQCInits
BulkQCMemo
BulkLotNo
AmpRetains
AmpDate
AmpPreLabel
AmpNumberGood
AmpNumberBad
AmpBulkRemain
AmpTimeIn
AmpTimeOut
AmpInits
QCStatus
QCPassFail
QCAuthInits
QCMemo
QCMethod
LotNo
QCExpMonth
QCExpDate
QCSellBy
QCFullValidation
QCValidation
FgInvCount
FgAccpacNote
CofAHeaderNo
QCChromatogram
Correct
DoNotCorrect
DensityUnit
DensityTemp
recipe

### groupBy `PfBatchID`

In [29]:
df_by_id = df_metrics_test.groupby('ProductNo')

---

In [30]:
# slice and dice your DataFrame
df_metrics_test.loc[('2020-02-14', 1.0)]

Unnamed: 0 ProductID      ProductNo  \
QCDate     ProductLevel                                        
2020-02-14 1.0                12224    108963   CUS-00000171   
           1.0                12688    113787   CUS-00002470   
           1.0                12690    113843   CUS-00002497   
           1.0                12704    113869   CUS-00002515   
           1.0                23094     55370      CUS-13853   
           1.0                24019     58659      CUS-14178   
           1.0                27116     71452      CUS-15490   
           1.0                29913     86059      CUS-19563   
           1.0                37079      1715       CUS-4698   
           1.0                56141      9212  G1969-85026-X   
           1.0                56148      9212  G1969-85026-X   
           1.0                59140    113789   ICUS-0000978   
           1.0                59903     25516      ICUS-2018   
           1.0                61355     56001      ICUS-3395   
           1.0                90422     98990      SOR-120-1   

                                          ProductName  LineID Description  \
QCDate     ProductLevel                                                     
2020-02-14 1.0                        Custom Standard     7.0         NaN   
           1.0                        Custom Standard     7.0         NaN   
           1.0                        Custom Standard     7.0         NaN   
           1.0                        Custom Standard     7.0         NaN   
           1.0                        Custom Standard     7.0         NaN   
           1.0                        Custom Standard     7.0         NaN   
           1.0                        Custom Standard     7.0         NaN   
           1.0                        Custom Standard     7.0         NaN   
           1.0                        Custom Standard     7.0         NaN   
           1.0                       Flushing Solvent    20.0         NaN   
           1.0                       Flushing Solvent    20.0         NaN   
           1.0                        Custom Standard     8.0         NaN   
           1.0                        Custom Standard     8.0         NaN   
           1.0                        Custom Standard     8.0         NaN   
           1.0           Oregon Pesticides Standard 3     7.0         NaN   

                        Specials OriginationDate                  EntryDate  \
QCDate     ProductLevel                                                       
2020-02-14 1.0               NaN      2019-04-11 2019-04-11 00:00:00.000000   
           1.0               NaN      2020-01-17 2020-01-17 00:00:00.000000   
           1.0               NaN      2020-01-22 2020-01-22 00:00:00.000000   
           1.0               NaN      2020-01-23 2020-01-23 00:00:00.000000   
           1.0               NaN      2012-03-01 2012-03-01 00:00:00.000000   
           1.0               NaN      2012-07-18 2012-07-18 00:00:00.000000   
           1.0               NaN      2014-03-26 2014-03-26 00:00:00.000000   
           1.0               NaN      2016-02-11 2016-02-11 00:00:00.000000   
           1.0               NaN      2003-04-30 2003-04-30 00:00:00.000000   
           1.0               NaN      2004-07-12 2004-07-12 00:00:00.000000   
           1.0               NaN      2004-07-12 2004-07-12 00:00:00.000000   
           1.0               NaN      2020-01-17 2020-01-17 13:37:20.626666   
           1.0               NaN      2007-06-05 2009-04-30 00:00:00.000000   
           1.0               NaN      2012-04-03 2012-04-03 00:00:00.000000   
           1.0               NaN      2017-08-25 2017-08-25 00:00:00.000000   

                        Status  ... QCValidation  FgInvCount  \
QCDate     ProductLevel         ...                            
2020-02-14 1.0             NaN  ...          1.0        10.0   
           1.0             NaN  ...          1.0        10.0   
           1.0            

In [31]:
df_by_id.describe().head()

Unnamed: 0                                                       \
                   count       mean       std   min    25%   50%    75%   max   
ProductNo                                                                       
01080-68703         18.0  13.555556  5.436502   5.0   9.25  13.5  17.75  23.0   
01080-68704-1       28.0  38.857143  8.712334  25.0  31.75  38.5  46.25  54.0   
051111BY-1           2.0  56.500000  0.707107  56.0  56.25  56.5  56.75  57.0   
051111BY-2           1.0  58.000000       NaN  58.0  58.00  58.0  58.00  58.0   
051117-560           1.0  59.000000       NaN  59.0  59.00  59.0  59.00  59.0   

              LineID       ... DensityUnit     DensityTemp                   \
               count mean  ...         75% max       count mean std min 25%   
ProductNo                  ...                                                
01080-68703     18.0  7.0  ...         NaN NaN         0.0  NaN NaN NaN NaN   
01080-68704-1   28.0  7.0  ...         NaN NaN         0.0  NaN NaN NaN NaN   
051111BY-1       2.0  7.0  ...         NaN NaN         0.0  NaN NaN NaN NaN   
051111BY-2       1.0  7.0  ...         NaN NaN         0.0  NaN NaN NaN NaN   
051117-560       1.0  7.0  ...         NaN NaN         0.0  NaN NaN NaN NaN   

                           
              50% 75% max  
ProductNo                  
01080-68703   NaN NaN NaN  
01080-68704-1 NaN NaN NaN  
051111BY-1    NaN NaN NaN  
051111BY-2    NaN NaN NaN  
051117-560    NaN NaN NaN  

[5 rows x 208 columns]